In [3]:
import pandas as pd
import os

In [4]:
dataset_folder = "./data/sentencedata"
destination_csv_path = "./data/sentencedata.csv"

In [17]:
def get_sentences_and_labels(article_id,directory=None):
    assert(directory is not None)
    
    article_file_path = os.path.join(directory, "article" + str(article_id) + ".txt")
    sentence_labels_file = os.path.join(directory, "article" + str(article_id) + ".task2.labels")
    
    article_sentences = []
    with open(article_file_path, "r", encoding="utf8") as article:
        for sent in article:
            article_sentences.append(str.strip(sent))
    
    sentence_labels = []
    sentence_ids = []
    with open(sentence_labels_file, "r") as labels:
        for label in labels:
            parts = label.split()
            sentence_ids.append(str.strip(parts[1]))
            sentence_labels.append(str.strip(parts[-1]))
            
    return {"aid" : article_id, "sentences" : article_sentences, "labels" : sentence_labels, "sids" : sentence_ids}


test = get_sentences_and_labels(111111112, directory = dataset_folder)

print("Number of Sentences:", len(test["sentences"]))
print("Number of Labels:", len(test["labels"]))

Number of Sentences: 50
Number of Labels: 50


In [18]:
#Get a list of all article ids
article_ids = [int(f[7:-4]) for f in os.listdir(dataset_folder) if f[-4:] == ".txt"]
print(len(article_ids))

293


In [19]:
final_result = dict()
for curr_id in article_ids:
    curr_result = get_sentences_and_labels(curr_id,directory=dataset_folder)
    
    if len(curr_result["sentences"])!=len(curr_result["labels"]):
        print("Mismatch on ID",curr_id,"! Skipping...")
        continue
    
    curr_result_filtered = [sl for sl in zip(curr_result["sentences"], curr_result["sids"], curr_result["labels"])]
    
    final_result[curr_result["aid"]] = dict()
    final_result[curr_result["aid"]]["scount_orig"] = len(curr_result["sentences"])
    final_result[curr_result["aid"]]["scount_new"] = len(curr_result_filtered)
    final_result[curr_result["aid"]]["data"] = curr_result_filtered
    
print(len(final_result.keys()))
final_result[111111112]["data"]

293


[('US bloggers banned from entering UK', '1', 'non-propaganda'),
 ('', '2', 'non-propaganda'),
 ('Two prominent US bloggers have been banned from entering the UK, the Home Office has said.',
  '3',
  'non-propaganda'),
 ('', '4', 'non-propaganda'),
 ('Pamela Geller and Robert Spencer co-founded anti-Muslim group Stop Islamization of America.',
  '5',
  'propaganda'),
 ('', '6', 'non-propaganda'),
 ('They were due to speak at an English Defence League march in Woolwich, where Drummer Lee Rigby was killed.',
  '7',
  'non-propaganda'),
 ('', '8', 'non-propaganda'),
 ('A government spokesman said individuals whose presence "is not conducive to the public good" could be excluded by the home secretary.',
  '9',
  'non-propaganda'),
 ('', '10', 'non-propaganda'),
 ('He added: "We condemn all those whose behaviours and views run counter to our shared values and will not stand for extremism in any form."',
  '11',
  'propaganda'),
 ('', '12', 'non-propaganda'),
 ("'Right decision'", '13', 'non

In [20]:
#Transform to list
sentences_all_meta = []
for aid in final_result.keys():
    article_sentences = final_result[aid]["data"]
    for curr_sent in article_sentences:
        sentences_all_meta.append((aid,curr_sent[0], curr_sent[1], curr_sent[2]))
print(len(sentences_all_meta))

15170


In [39]:
sentences = pd.DataFrame.from_records(sentences_all_meta,columns=["aid", "sentence", "sid", "flag"])
sentences = sentences[sentences.sentence != ""]
sentences = sentences[sentences.sentence != "---"]
sentences = sentences[sentences.sentence != "..."]
sentences.head()

,aid,sentence,sid,flag
0,111111112,US bloggers banned from entering UK,1,non-propaganda
2,111111112,Two prominent US bloggers have been banned fro...,3,non-propaganda
4,111111112,Pamela Geller and Robert Spencer co-founded an...,5,propaganda
6,111111112,They were due to speak at an English Defence L...,7,non-propaganda
8,111111112,A government spokesman said individuals whose ...,9,non-propaganda


In [40]:
sentences.shape

(14258, 4)